In [ ]:
!apt-get -qq install subversion
!pip install kor
!pip install einops auto-gptq

!pip install sentence_transformers
!pip -q install  tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers
!pip  install langchain
!pip install safetensors

debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package libapr1:amd64.
(Reading database ... 19391 files and directories currently installed.)
Preparing to unpack .../0-libapr1_1.7.0-8ubuntu0.22.04.1_amd64.deb ...
Unpacking libapr1:amd64 (1.7.0-8ubuntu0.22.04.1) ...
Selecting previously unselected package libaprutil1:amd64.
Preparing to unpack .../1-libaprutil1_1.6.1-5ubuntu4.22.04.1_amd64.deb ...
Unpacking libaprutil1:amd64 (1.6.1-5ubuntu4.22.04.1) ...
Selecting previously unselected package libserf-1-1:amd64.
Preparing to unpack .../2-libserf-1-1_1.3.9-10ubuntu2_amd64.deb ...
Unpacking libserf-1-1:amd64 (1.3.9-10ubuntu2) ...
Selecting previously unselected package libutf8proc2:amd64.
Preparing to unpack .../3-libutf8proc2_2.7.0-3_amd64.deb ...
Unpacking libutf8proc2:amd64 (2.7.0-3) ...
Selecting previously unselected package libsvn1:amd64.
Preparing to unpack .../4-libsvn1_1.14.1-3ubuntu0.22.04.1_amd64.deb ...
Unpacking libsvn

# Get podcast data from gkamradt github and Install ChatOpenLLM

In [ ]:
!svn checkout https://github.com/gkamradt/langchain-tutorials/trunk/data/Transcripts
# Use your paths
transcript_paths = [
    './Transcripts/MFMPod/mfm_pod_steph.txt',
    './Transcripts/MFMPod/mfm_pod_alex.txt',
    './Transcripts/MFMPod/mfm_pod_rob.txt'
]

# Open and read a file
with open('./Transcripts/MFMPod/mfm_pod_steph.txt') as file:
    transcript = file.read()


A    Transcripts/MFMPod
A    Transcripts/MFMPod/mfm_pod_alex.txt
A    Transcripts/MFMPod/mfm_pod_rob.txt
A    Transcripts/MFMPod/mfm_pod_steph.txt
A    Transcripts/acme_co_v2.txt
Checked out revision 102.


In [ ]:
# LangChain basics
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import create_extraction_chain
from langchain.vectorstores import Chroma

# Vector Store and retrievals
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

# Chat Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Supporting libraries
import os
from dotenv import load_dotenv

load_dotenv()


False

# Using ChatOpenLLM for Vicuna-33B-1-1-preview-SuperHOT-8K-GPTQ




In [ ]:
!git clone https://github.com/Shafi2016/ChatOpenLLM.git
%cd ./ChatOpenLLM

!pip install .



fatal: destination path 'ChatOpenLLM' already exists and is not an empty directory.
/workspace/ChatOpenLLM
Processing /workspace/ChatOpenLLM
  Preparing metadata (setup.py) ... done
  Created wheel for ChatOpenLLM: filename=ChatOpenLLM-0.1-py3-none-any.whl size=5691 sha256=2e249438273d27e62df32a76b880c36dbe31a0487b9c083ca61d0e63daa054bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-akr0f3wh/wheels/0b/93/ad/c721b583a087b59b6ae941f26349eaaa0f0977222d074e127f
Successfully built ChatOpenLLM


In [ ]:
%cd ..

/workspace


In [ ]:
from ChatOpenLLM.model import ChatGPTQ
import torch

llm = ChatGPTQ(
    model_name_or_path="TheBloke/Vicuna-33B-1-1-preview-SuperHOT-8K-GPTQ",
    model_basename="vicuna-33b-1.3-preview-superhot-8k-GPTQ-4bit--1g.act.order",
    gen_kwargs=dict(temperature=0, top_p=0.95, repetition_penalty=1.15),
    use_safetensors=True,
    #load_in_4bit=True,
    trust_remote_code=True,
    device_map='auto',
    use_triton=False,
    quantize_config=None,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True )


A new version of the following files was downloaded from https://huggingface.co/TheBloke/Vicuna-33B-1-1-preview-SuperHOT-8K-GPTQ:
- modelling_llama.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The safetensors archive passed at /root/.cache/huggingface/hub/models--TheBloke--Vicuna-33B-1-1-preview-SuperHOT-8K-GPTQ/snapshots/e724fd3a9133c107049134905e049ab7acf14352/vicuna-33b-1.3-preview-superhot-8k-GPTQ-4bit--1g.act.order.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


In [ ]:
print(transcript[:280])

Shaan Puri (0:00:00-0:00:03): D to see hearing AIDS. I think that's actually going to be a big deal. 

Sam Parr (0:00:03-0:00:05): And they're profitable. 

Shaan Puri (0:00:05-0:00:08): I mean, I'm just turning you on. Yeah, they were. 

Sam Parr (0:00:12-0:00:13): They Mormon. 


# HuggingFaceEmbeddings

In [ ]:

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/e5-large-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)



In [ ]:
# Load up your text splitter
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " "], chunk_size=10000, chunk_overlap=2200)

# I'm only doing the first 23250 characters. This to save on costs. When you're doing your exercise you can remove this to let all the data through
transcript_subsection_characters = 23250
docs = text_splitter.create_documents([transcript[:transcript_subsection_characters]])
print (f"You have {len(docs)} docs. First doc is {llm.get_num_tokens(docs[0].page_content)} tokens")

Token indices sequence length is longer than the specified maximum sequence length for this model (2876 > 1024). Running this sequence through the model will result in indexing errors


You have 3 docs. First doc is 2876 tokens


# Extract Topic Titles & Short Description

In [ ]:
template="""
You are a helpful assistant that helps retrieve topics talked about in a podcast transcript
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics include:
  - Themes
  - Business Ideas
  - Interesting Stories
  - Money making businesses
  - Quick stories about people
  - Mental Frameworks
  - Stories about an industry
  - Analogies mentioned
  - Advice or words of caution
  - Pieces of news or current events
- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the podcast
- Do not respond with anything outside of the podcast. If you don't see any topics, say, 'No Topics'
- Do not respond with numbers, just bullet points
- Do not include anything about 'Marketing Against the Grain'
- Only pull topics from the transcript. Do not use the examples
- Make your titles descriptive but concise. Example: 'Shaan's Experience at Twitch' should be 'Shaan's Interesting Projects At Twitch'
- A topic should be substantial, more than just a one-off comment

% START OF EXAMPLES
 - Sam’s Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
 - Shaan’s Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
 - Revenge Against The Spam Calls: A couple of businesses focused on protecting consumers: RoboCall, TrueCaller, DoNotPay, FitIt
 - Wildcard CEOs vs. Prudent CEOs: However, Munger likes to surround himself with prudent CEO’s and says he would never hire Musk.
 - Chess Business: Priyav, a college student, expressed his doubts on the MFM Facebook group about his Chess training business, mychesstutor.com, making $12.5K MRR with 90 enrolled.
 - Restaurant Refiller: An MFM Facebook group member commented on how they pay AirMark $1,000/month for toilet paper and toilet cover refills for their restaurant. Shaan sees an opportunity here for anyone wanting to compete against AirMark.
 - Collecting: Shaan shared an idea to build a mobile only marketplace for a collectors’ category; similar to what StockX does for premium sneakers.
% END OF EXAMPLES
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

In [ ]:
template="""
You are a helpful assistant that helps retrieve topics talked about in a podcast transcript
- You will be given a series of bullet topics of topics vound
- Your goal is to exract the topic names and brief 1-sentence description of the topic
- Deduplicate any bullet points you see
- Only pull topics from the transcript. Do not use the examples

% START OF EXAMPLES
 - Sam’s Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
 - Shaan’s Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
% END OF EXAMPLES
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_combine = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

# The First Pass : Extract the topics per custom prompts

In [ ]:

# Assuming `chat_model` is your LLM model instance
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,
                             combine_prompt=chat_prompt_combine)


In [ ]:
topics_found = chain.run({"input_documents": docs})
print (topics_found)

Token indices sequence length is longer than the specified maximum sequence length for this model (3894 > 2048). Running this sequence through the model will result in indexing errors


Working at Twitch and Embracing Interesting Opportunities
Shaan Puri discussed his past experience working at Twitch, embracing interesting opportunities like buying a company called Stream Hive, which turned out to be a successful decision.

Making Friends with Emmet at Twitch
Shaan Puri mentioned befriending Emmet at Twitch, who helped him realize the potential of live streaming and the growing industry around it.

Ninja Poached and Staying in Meetings
Sam shared the story of Ninja, a popular streamer, getting poached by Microsoft, and the importance of staying in meetings and attending events to build connections.

Talent Level at Andreessen Horowitz
The conversation touched on the high talent level found at Andreessen Horowitz, with notable people like Marc Andreessen and Emmet who have contributed significantly to the firm's success.

Commercial Real Estate vs. Fractional Real Estate
The podcast explored the difference between commercial real estate, which is traditionally used fo

In [ ]:
# Save the string to a text file
with open('topics_found.txt', 'w', encoding='utf-8') as f:
    f.write(topics_found)


# Using Kor, Turn LLM output into structured data

In [ ]:
from kor.nodes import Object, Text

schema = Object(
    id="topic_info",
    description="Information about a given topic.",
    attributes=[
        Text(
            id="topic_name",
            description="The title of the topic listed",
        ),
        Text(
            id="description",
            description="The description of the topic listed",
        ),
        Text(
            id="tag",
            description="The type of content being described",
            enum=['Business Models', 'Life Advice', 'Health & Wellness', 'Stories'],
        ),
    ],
    many=True,
)


In [ ]:
from kor.extraction import create_extraction_chain
from kor import extract_from_documents, from_pydantic, create_extraction_chain
from kor.nodes import Object, Text, Number


In [ ]:

chain = create_extraction_chain(llm,schema)
topics_structured = chain.run(topics_found)
topics_structured

'topic_info: [\n{\n"topic_name": "Working at Twitch",\n"description": "Shaan Puri\'s experience working at Twitch",\n"tag": "company"\n},\n{\n"topic_name": "Embracing Interesting Opportunities",\n"description": "Shaan Puri embracing interesting opportunities like buying Stream Hive",\n"tag": "strategy"\n},\n{\n"topic_name": "Making Friends with Emmet",\n"description": "Shaan Puri befriending Emmet at Twitch",\n"tag": "networking"\n},\n{\n"topic_name": "Ninja Poached",\n"description": "Ninja, a popular streamer, getting poached by Microsoft",\n"tag": "industry_event"\n},\n{\n"topic_name": "Talent Level at Andreessen Horowitz",\n"description": "High talent level at Andreessen Horowitz",\n"tag": "people"\n},\n{\n"topic_name": "Commercial Real Estate vs. Fractional Real Estate",\n"description": "Difference between commercial and fractional real estate",\n"tag": "property_type"\n},\n{\n"topic_name": "Temple Immersive",\n"description": "Temple Immersive in San Francisco example of a nightclu

In [ ]:
print(topics_structured)

topic_info: [
{
"topic_name": "Working at Twitch",
"description": "Shaan Puri's experience working at Twitch",
"tag": "company"
},
{
"topic_name": "Embracing Interesting Opportunities",
"description": "Shaan Puri embracing interesting opportunities like buying Stream Hive",
"tag": "strategy"
},
{
"topic_name": "Making Friends with Emmet",
"description": "Shaan Puri befriending Emmet at Twitch",
"tag": "networking"
},
{
"topic_name": "Ninja Poached",
"description": "Ninja, a popular streamer, getting poached by Microsoft",
"tag": "industry_event"
},
{
"topic_name": "Talent Level at Andreessen Horowitz",
"description": "High talent level at Andreessen Horowitz",
"tag": "people"
},
{
"topic_name": "Commercial Real Estate vs. Fractional Real Estate",
"description": "Difference between commercial and fractional real estate",
"tag": "property_type"
},
{
"topic_name": "Temple Immersive",
"description": "Temple Immersive in San Francisco example of a nightclub transforming into a yoga studio",

In [ ]:
import pandas as pd

topic_info = [
    {
        "topic_name": "Working at Twitch",
        "description": "Shaan Puri's experience working at Twitch",
        "tag": "company"
    },
    {
        "topic_name": "Embracing Interesting Opportunities",
        "description": "Shaan Puri embracing interesting opportunities like buying Stream Hive",
        "tag": "strategy"
    },
    {
        "topic_name": "Making Friends with Emmet",
        "description": "Shaan Puri befriending Emmet at Twitch",
        "tag": "networking"
    },
    {
        "topic_name": "Ninja Poached",
        "description": "Ninja, a popular streamer, getting poached by Microsoft",
        "tag": "industry_event"
    },
    {
        "topic_name": "Talent Level at Andreessen Horowitz",
        "description": "High talent level at Andreessen Horowitz",
        "tag": "people"
    },
    {
        "topic_name": "Commercial Real Estate vs. Fractional Real Estate",
        "description": "Difference between commercial and fractional real estate",
        "tag": "property_type"
    },
    {
        "topic_name": "Temple Immersive",
        "description": "Temple Immersive in San Francisco example of a nightclub transforming into a yoga studio",
        "tag": "unique_space"
    },
    {
        "topic_name": "Fractional Real Estate for Gyms",
        "description": "Fractional real estate for gyms",
        "tag": "property_use"
    },
    {
        "topic_name": "Rage Rooms",
        "description": "Rage rooms as a growing trend in self-expression and entertainment",
        "tag": "trend"
    },
    {
        "topic_name": "Hampton's Raleigh Williams",
        "description": "Raleigh Williams' 1984 Bluebird bus escape room business success story",
        "tag": "success_story"
    }
]

df = pd.DataFrame(topic_info)
df.to_csv('topics.csv', index=False)

print(df.head(5))


                            topic_name  \
0                    Working at Twitch   
1  Embracing Interesting Opportunities   
2            Making Friends with Emmet   
3                        Ninja Poached   
4  Talent Level at Andreessen Horowitz   

                                         description             tag  
0          Shaan Puri's experience working at Twitch         company  
1  Shaan Puri embracing interesting opportunities...        strategy  
2             Shaan Puri befriending Emmet at Twitch      networking  
3  Ninja, a popular streamer, getting poached by ...  industry_event  
4           High talent level at Andreessen Horowitz          people  


# Expand on the topics with Chroma DB Multi Doc Retriever LangChain

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=800)

docs = text_splitter.create_documents([transcript[:transcript_subsection_characters]])

print (f"You have {len(docs)} docs. First doc is {llm.get_num_tokens(docs[0].page_content)} tokens")

You have 8 docs. First doc is 807 tokens


In [ ]:
persist_directory = 'db'

embedding = hf

vectordb = Chroma.from_documents(documents=docs,
                                 embedding=embedding,
                                 persist_directory=persist_directory)
# persiste the db to disk
vectordb.persist()
vectordb = None
# Now we can load the persisted database from disk, and use it as normal.

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

In [ ]:
# The system instructions. Notice the 'context' placeholder down below. This is where our relevant docs will go.
# The 'question' in the human message below won't be a question per se, but rather a topic we want to get relevant information on
system_template = """
You will be given text from a podcast transcript which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]

# This will pull the two messages together and get them ready to be sent to the LLM through the retriever
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 chain_type_kwargs = {
#                                      'verbose': True,
                                     'prompt': CHAT_PROMPT
                                 })

In [ ]:
import pandas as pd

# Read CSV file
df = pd.read_csv("./topics.csv")

# If you want to see the first 5 rows of the data frame, uncomment the next line
# print(df.head())

# Iterate over the first 5 rows of the data frame
for index, row in df.head().iterrows():
    query = f"{row['topic_name']}: {row['description']}"

    # I'm assuming that 'qa' is a predefined object and 'run' is its method which takes a query and returns the answer.
    # Replace this line with the actual method you're using to get the expanded topic.
    expanded_topic = qa.run(query)

    print(f"{row['topic_name']}: {row['description']}")
    print(expanded_topic)
    print("\n\n")


Working at Twitch: Shaan Puri's experience working at Twitch
Shaan Puri had an interesting experience working at Twitch, as mentioned in the podcast excerpt. He discussed how he approached his job, focusing on finding the most interesting things happening within the company and attending meetings with the goal of creating intersections between himself and others, rather than being solely focused on doing his actual job. He mentioned that he was able to work with Emmet, whom he considered the most interesting person in the building, and stayed in meetings to hang out and learn from these interactions. Shaan also mentioned that he wrote a plan for Ninja's situation and attended a meeting at 8:00 pm, which was outside of his normal work hours, to be involved in the decision-making process. This approach allowed him to learn and interact with the smart people at Twitch, making his job more valuable than just a paycheck.


While at Twitch, Shaan also mentioned how the top streamer on the pl

# With Timestamps

In [ ]:
system_template = """
What is the first timestamp when the speakers started talking about a topic the user gives?
Only respond with the timestamp, nothing else. Example: 0:18:24
----------------
{context}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever= retriever,
                                 chain_type_kwargs = {
#                                      'verbose': True,
                                     'prompt': CHAT_PROMPT
                                 })



In [ ]:
import pandas as pd

# Read CSV file
df = pd.read_csv("./topics.csv")

# Initialize an empty list to hold timestamps
topic_timestamps = []

# Iterate over all rows of the data frame
for index, row in df.iterrows():
    query = f"{row['topic_name']} - {row['description']}"

    # I'm assuming that 'qa' is a predefined object and 'run' is its method which takes a query and returns the timestamp.
    # Replace this line with the actual method you're using to get the timestamp.
    timestamp = qa.run(query)

    topic_timestamps.append(f"{timestamp} - {row['topic_name']}")

# Now 'topic_timestamps' is a list of strings, each string is in the format "timestamp - topic_name"
# For example: ["2023-06-29 14:00 - Topic 1", "2023-06-29 14:05 - Topic 2", ...]


In [ ]:
print ("\n".join(sorted(topic_timestamps)))

16:22: Rage rooms have emerged as an intriguing and fascinating trend in the realm of self-expression and entertainment, providing individuals with a unique outlet to release stress and emotions in a controlled environment.

21:15: The concept of rage rooms is increasingly gaining popularity, with people interested in smashing objects to relieve tension and emotions.

23:35: Rage rooms can be seen as a form of alternative entertainment, combining excitement and release in a safe environment.

32:55: We might see more rage room pop-ups or mobile rage rooms catering to different customer preferences and scenarios in the future.

42:48: Rage rooms might not be a traditional form of therapy, but they can help people let go of anger or stress in a controlled way.

52:37: There might be a growing market for rage rooms, especially if they are combined with other experiences like yoga or alcohol.

59:45: Rage rooms could be an interesting investment opportunity for someone looking to diversify